In [3]:
import os
import pandas as pd

In [2]:
# Je me positionne dans mon répertoire de travail personnel
os.chdir("C:\\Users\\Julien Gremillot\\OneDrive\\Documents\\" +
         "OpenClassrooms\\github\\" +
         "OpenClassrooms_Ingenieur_Machine_Learning\\" +
         "Catégorisez automatiquement des questions")

In [11]:
# Lecture du CSV
df = pd.read_csv('txt_and_tags.csv')
# petit contournement de l'erreur d'avoir sauvegardé en texte
df['txt'] = [txt.replace("['", "").replace("']", "").split("', '") for txt in df['txt']]
df['tags_filtered'] = [txt.replace("['", "").replace("']", "").split("', '") for txt in df['tags_filtered']]

In [12]:
df

,txt,tags_filtered
0,"[convert, decimal, double, c, want, use, track...",[c#]
1,"[width, collapse, percentage, width, child, el...","[css, html]"
2,"[calculate, someone, age, based, datetime, typ...","[c#, .net]"
3,"[calculate, relative, time, c, given, specific...",[c#]
4,"[binary, data, mysql, store, binary, data, mysql]","[mysql, database]"
...,...,...
139037,"[access, n, th, element, array, php, embarrass...",[php]
139038,"[c, vector, int, size_type, variable, point, d...",[c++]
139039,"[get, code, coverage, report, using, jest, way...",[javascript]
139040,"[unable, type, visual, studio, unable, edit, f...","[c#, visual-studio, asp.net]"


In [13]:
df['txt'][0][0]

'convert'

In [14]:
df['tags_filtered'][0][0]

'c#'